In [1]:
# coding=utf-8
# @author:bryan
# blog: https://blog.csdn.net/bryan__
# github: https://github.com/YouChouNoBB/2018-tencent-ad-competition-baseline
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import os

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
ad_feature=pd.read_csv('./Oridata/tc2018/adFeature.csv')

In [4]:
ad_feature.head()

,aid,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType
0,177,8203,76104,1500666,59,282,0,6
1,2050,19441,178687,245165,53,1,0,6
2,1716,5552,158101,1080850,35,27,113,9
3,336,370,4833,119845,22,67,113,9
4,671,45705,352827,660519,42,67,0,4


In [5]:
ad_feature.shape

(173, 8)

In [6]:
if os.path.exists('./Oridata/tc2018/userFeature.csv'):
    user_feature=pd.read_csv('./Oridata/tc2018/userFeature.csv')
else:
    print('No user_feature')
    userFeature_data = []
    with open('./Oridata/tc2018/userFeature.data', 'r') as f:
        for i, line in enumerate(f):
            line = line.strip().split('|')
            userFeature_dict = {}
            for each in line:
                each_list = each.split(' ')
                userFeature_dict[each_list[0]] = ' '.join(each_list[1:])
            userFeature_data.append(userFeature_dict)
            if i % 100000 == 0:
                print(i)
        user_feature = pd.DataFrame(userFeature_data)
        user_feature.to_csv('../data/userFeature.csv', index=False)
        del userFeature_data

In [7]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
user_feature.head()

,LBS,age,appIdAction,appIdInstall,carrier,consumptionAbility,ct,education,gender,house,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,marriageStatus,os,topic1,topic2,topic3,uid
0,950.0,4,NaN,NaN,1,2,3 1,7,2,NaN,93 70 77 86 109 47 75 69 45 8 29 49 83 6 46 36...,46 19 13 29,NaN,NaN,52 100 72 131 116 11 71 12 8 113 28 73 6 132 9...,664359 276966 734911 103617 562294,11395 79112 115065 77033 36176,NaN,11,2,9826 105 8525 5488 7281,9708 5553 6745 7477 7150,NaN,26325489
1,803.0,2,NaN,NaN,1,1,3 1,2,1,NaN,75 29,33,NaN,NaN,NaN,338851 361151 542834 496283 229952,80263 39618 53539 180 38163,NaN,5 13,1,4391 9140 5669 1348 4388,9401 7724 1380 8890 7153,NaN,1184123
2,927.0,1,NaN,NaN,3,1,3 1,5,1,NaN,70 12 28 106 59 49 41 6 42 115 35 116 36 11 96,51 22 79 81 70 6 21 4 41 35,NaN,NaN,77 72 80 116 101 13 1 109 8 50 6 42 76 9 46 36...,746140 695808 126355 771775 411511,105115 71378 41409 74061 44005,NaN,13 10,1,1502 5488 9826 2187 8088,5005 9154 2756 5612 4209,NaN,76072711
3,486.0,4,NaN,NaN,3,1,1 3,7,1,NaN,93 70 77 47 76 75 69 29 59 49 122 42 46 23 17 ...,24 70 19 13 30 10,NaN,NaN,100 80 92 37 116 13 47 4 71 8 50 28 98 115 6 4...,283399 402245 734509 654027 32061,74010 32918 67882 116802 20957,NaN,11,1 2,1619 7342 3064 9213 8525,810 2438 5659 1844 9262,NaN,63071413
4,112.0,5,NaN,NaN,3,2,3 1 4 2,6,1,NaN,75 29 49 122 36 11,NaN,NaN,NaN,131 116 13 8 6 132 42 9 59 18 58 64 129 103,562294 157603 589706 657719 495672,62764 97803 89066 55545 74061,NaN,11,1,477 9826 5808 644 2747,5483 2199 5424 1511 7751,NaN,81294159


In [8]:
user_feature.shape

(11420039, 24)

In [9]:
list(user_feature) #打印每列的名字

['LBS',
 'age',
 'appIdAction',
 'appIdInstall',
 'carrier',
 'consumptionAbility',
 'ct',
 'education',
 'gender',
 'house',
 'interest1',
 'interest2',
 'interest3',
 'interest4',
 'interest5',
 'kw1',
 'kw2',
 'kw3',
 'marriageStatus',
 'os',
 'topic1',
 'topic2',
 'topic3',
 'uid']

In [10]:
train=pd.read_csv('./Oridata/tc2018/train.csv')
predict=pd.read_csv('./Oridata/tc2018/test1.csv')

In [11]:
train[:4]

,aid,uid,label
0,699,78508957,0
1,1991,3637295,0
2,1119,19229018,0
3,2013,79277120,0


In [12]:
train.shape

(8798814, 3)

In [13]:
predict.head()

,aid,uid
0,2118,64355836
1,692,45051997
2,692,10869198
3,1918,75929554
4,1596,5790162


In [ ]:
predict.shape

(2265989, 2)

In [ ]:
a = train.loc[train['label']==-1]

In [ ]:
a.head()

,aid,uid,label


In [ ]:
b = train.loc[train['label']==-1,'label']

In [ ]:
b.head()

Series([], Name: label, dtype: int64)

In [ ]:
train.loc[train['label']==-1,'label']=0
predict['label']=-1 # 运行有点慢

In [ ]:
data=pd.concat([train,predict])

In [ ]:
data.shape

(11064803, 3)

In [ ]:
data.head()

,aid,uid,label
0,699,78508957,0
1,1991,3637295,0
2,1119,19229018,0
3,2013,79277120,0
4,692,41528441,0


In [ ]:
data=pd.merge(data,ad_feature,on='aid',how='left')

In [ ]:
data=pd.merge(data,user_feature,on='uid',how='left') 

In [ ]:
data.head()

,aid,uid,label,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType,LBS,age,appIdAction,appIdInstall,carrier,consumptionAbility,ct,education,gender,house,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,marriageStatus,os,topic1,topic2,topic3
0,699,78508957,0,1082,295940,731679,59,13,0,6,576.0,1,NaN,NaN,0,1,0,1,1,NaN,70 100 48 49 23 36 11,NaN,NaN,NaN,NaN,124532 762727 771871 8561 610048,NaN,NaN,10,0,4908 9745 7070 4203 3292,NaN,NaN
1,1991,3637295,0,702,42104,1441131,53,10,4669,11,353.0,3,NaN,NaN,1,0,3,7,2,NaN,93 109 75 29 59 49 6 5 73 46 22 36 11 3,NaN,NaN,NaN,100 131 37 8 46,531499 542834 303322 503318 68211,89232 30101 117462 117548 85039,NaN,13,1,8445 6762 7608 523 7688,4518 862 3643 702 5247,NaN
2,1119,19229018,0,3993,63752,798752,59,10,19256,11,94.0,4,NaN,NaN,0,0,0,1,1,NaN,93 37 47 75 29 46 17 9 56 36 11,NaN,NaN,NaN,NaN,255920 190937 576486 460368 135689,NaN,NaN,13 10,0,941 4674 9735 8953 5352,NaN,NaN
3,2013,79277120,0,6937,186348,1427984,35,89,3791,9,48.0,2,NaN,NaN,1,1,3 1,2,1,1.0,49 18,42 9 1 46 70 14 29 10 35,NaN,NaN,92 116 128 42 5 121 64,101914 294138,57664 32928 40670 11395 79112,NaN,10,1,4538 6790 8850 3422 1940,2979 1877 358 666 4166,NaN
4,692,41528441,0,6946,296367,455396,59,24,3794,11,333.0,1,NaN,NaN,1,1,1 3 4,7,1,NaN,70 109 12 75 76 106 29 59 49 41 111 85 17 56 1...,47 42 73 9 31 46 32 70 21 33 19 60 13 74 16 55...,NaN,NaN,100 72 37 11 4 79 113 6 46 59,155585 562294 112872 273650 371405,67951 40801 56571 119723 16159,NaN,10,1,4535 5590 47 5570 2078,856 4587 1877 392 9103,NaN


In [ ]:
data.shape

(11064803, 33)

In [ ]:
data=data.fillna('-1')

In [ ]:
data.head()

,aid,uid,label,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType,LBS,age,appIdAction,appIdInstall,carrier,consumptionAbility,ct,education,gender,house,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,marriageStatus,os,topic1,topic2,topic3
0,699,78508957,0,1082,295940,731679,59,13,0,6,576,1,-1,-1,0,1,0,1,1,-1,70 100 48 49 23 36 11,-1,-1,-1,-1,124532 762727 771871 8561 610048,-1,-1,10,0,4908 9745 7070 4203 3292,-1,-1
1,1991,3637295,0,702,42104,1441131,53,10,4669,11,353,3,-1,-1,1,0,3,7,2,-1,93 109 75 29 59 49 6 5 73 46 22 36 11 3,-1,-1,-1,100 131 37 8 46,531499 542834 303322 503318 68211,89232 30101 117462 117548 85039,-1,13,1,8445 6762 7608 523 7688,4518 862 3643 702 5247,-1
2,1119,19229018,0,3993,63752,798752,59,10,19256,11,94,4,-1,-1,0,0,0,1,1,-1,93 37 47 75 29 46 17 9 56 36 11,-1,-1,-1,-1,255920 190937 576486 460368 135689,-1,-1,13 10,0,941 4674 9735 8953 5352,-1,-1
3,2013,79277120,0,6937,186348,1427984,35,89,3791,9,48,2,-1,-1,1,1,3 1,2,1,1,49 18,42 9 1 46 70 14 29 10 35,-1,-1,92 116 128 42 5 121 64,101914 294138,57664 32928 40670 11395 79112,-1,10,1,4538 6790 8850 3422 1940,2979 1877 358 666 4166,-1
4,692,41528441,0,6946,296367,455396,59,24,3794,11,333,1,-1,-1,1,1,1 3 4,7,1,-1,70 109 12 75 76 106 29 59 49 41 111 85 17 56 1...,47 42 73 9 31 46 32 70 21 33 19 60 13 74 16 55...,-1,-1,100 72 37 11 4 79 113 6 46 59,155585 562294 112872 273650 371405,67951 40801 56571 119723 16159,-1,10,1,4535 5590 47 5570 2078,856 4587 1877 392 9103,-1


In [ ]:
one_hot_feature=['LBS','age','carrier','consumptionAbility','education','gender','house','os','ct','marriageStatus','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature=['appIdAction','appIdInstall','interest1','interest2','interest3','interest4','interest5','kw1','kw2','kw3','topic1','topic2','topic3']

In [ ]:
for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

In [ ]:
data.shape

(11064803, 33)

In [ ]:
data.head()

,aid,uid,label,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType,LBS,age,appIdAction,appIdInstall,carrier,consumptionAbility,ct,education,gender,house,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,marriageStatus,os,topic1,topic2,topic3
0,699,78508957,0,15,108,61,59,5,0,1,490,1,-1,-1,0,1,0,1,1,0,70 100 48 49 23 36 11,-1,-1,-1,-1,124532 762727 771871 8561 610048,-1,-1,1,0,4908 9745 7070 4203 3292,-1,-1
1,1991,3637295,0,9,38,135,53,4,14,3,299,3,-1,-1,1,0,33,7,2,0,93 109 75 29 59 49 6 5 73 46 22 36 11 3,-1,-1,-1,100 131 37 8 46,531499 542834 303322 503318 68211,89232 30101 117462 117548 85039,-1,6,1,8445 6762 7608 523 7688,4518 862 3643 702 5247,-1
2,1119,19229018,0,20,53,69,59,4,28,3,82,4,-1,-1,0,0,0,1,1,0,93 37 47 75 29 46 17 9 56 36 11,-1,-1,-1,-1,255920 190937 576486 460368 135689,-1,-1,7,0,941 4674 9735 8953 5352,-1,-1
3,2013,79277120,0,25,93,133,35,23,10,2,41,2,-1,-1,1,1,34,2,1,1,49 18,42 9 1 46 70 14 29 10 35,-1,-1,92 116 128 42 5 121 64,101914 294138,57664 32928 40670 11395 79112,-1,1,1,4538 6790 8850 3422 1940,2979 1877 358 666 4166,-1
4,692,41528441,0,26,109,37,59,8,11,3,283,1,-1,-1,1,1,10,7,1,0,70 109 12 75 76 106 29 59 49 41 111 85 17 56 1...,47 42 73 9 31 46 32 70 21 33 19 60 13 74 16 55...,-1,-1,100 72 37 11 4 79 113 6 46 59,155585 562294 112872 273650 371405,67951 40801 56571 119723 16159,-1,1,1,4535 5590 47 5570 2078,856 4587 1877 392 9103,-1


In [ ]:
train=data[data.label!=-1]
train_y=train.pop('label')
# train, test, train_y, test_y = train_test_split(train,train_y,test_size=0.2, random_state=2018)
test=data[data.label==-1]
res=test[['aid','uid']]
test=test.drop('label',axis=1)
enc = OneHotEncoder()

In [ ]:
train_x=train[['creativeSize']] # 这是广告牌大小，当然不能编码啦，否则失去意义
test_x=test[['creativeSize']]

In [ ]:
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a=enc.transform(train[feature].values.reshape(-1, 1)) 
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a))
    test_x = sparse.hstack((test_x, test_a))
print('one-hot prepared !')

one-hot prepared !


In [ ]:
cv=CountVectorizer()
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
    test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a))
    test_x = sparse.hstack((test_x, test_a))
print('cv prepared !')

cv prepared !


In [ ]:
def LGB_test(train_x,train_y,test_x,test_y):
    print("LGB test")
    clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
        max_depth=-1, n_estimators=1000, objective='binary',
        subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
        learning_rate=0.05, min_child_weight=50,random_state=2018,n_jobs=-1
    )
    clf.fit(train_x, train_y,eval_set=[(train_x, train_y),(test_x,test_y)],eval_metric='auc',early_stopping_rounds=100)
    # print(clf.feature_importances_)
    return clf,clf.best_score_[ 'valid_1']['auc']

In [ ]:
def LGB_predict(train_x,train_y,test_x,res):
    clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
        max_depth=-1, n_estimators=1500, objective='binary',
        subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
        learning_rate=0.05, min_child_weight=50, random_state=2018, n_jobs=-1
    )
    clf.fit(train_x, train_y, eval_set=[(train_x, train_y)], eval_metric='auc',early_stopping_rounds=100)
    res['score'] = clf.predict_proba(test_x)[:,1]
    res['score'] = res['score'].apply(lambda x: float('%.6f' % x))
    print("THE END!!")
    res.to_csv('./dataAfterProcess/submission.csv', index=False)
    os.system('zip ./dataAfterProcess/baseline.zip ./dataAfterProcess/submission.csv')
    return clf

In [ ]:
model=LGB_predict(train_x,train_y,test_x,res)